In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 46.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=1ae016c953ec2a8782f1e47ae2d955ad2f75984946f9ee3757c5fc8ee474d69b
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
import time
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('tree').getOrCreate()
df = spark.read.csv('/content/drive/MyDrive/Datasets/Thesis/New Dataset/dr16.csv', inferSchema=True, header=True)
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Unnamed: 0: integer (nullable = true)
 |-- modelMag_u: double (nullable = true)
 |-- modelMag_g: double (nullable = true)
 |-- modelMag_r: double (nullable = true)
 |-- modelMag_i: double (nullable = true)
 |-- modelMag_z: double (nullable = true)
 |-- fiberMag_u: double (nullable = true)
 |-- fiberMag_g: double (nullable = true)
 |-- fiberMag_r: double (nullable = true)
 |-- fiberMag_i: double (nullable = true)
 |-- fiberMag_z: double (nullable = true)
 |-- petroR50_r: double (nullable = true)
 |-- petroR90_r: double (nullable = true)
 |-- petroR50_z: double (nullable = true)
 |-- petroR90_z: double (nullable = true)
 |-- r: double (nullable = true)
 |-- i: double (nullable = true)
 |-- z: double (nullable = true)
 |-- mmug: double (nullable = true)
 |-- mmgr: double (nullable = true)
 |-- mmri: double (nullable = true)
 |-- mmiz: double (nullable = true)
 |-- mfug: double (nullable = true)
 |-- mfgr: double (nullable = true)
 |-- mfri: do

In [ ]:
import pyspark.sql.functions as func
df = df.withColumn("redshift", func.round(df["redshift"], 2).cast('integer'))
df.groupBy('redshift').count().show()

+--------+------+
|redshift| count|
+--------+------+
|       1| 48930|
|       3| 71008|
|       2| 31638|
|       0|409778|
|       6|  5347|
|       5|  6185|
|       4|  9264|
|       7|    81|
+--------+------+



In [ ]:
df = df.drop('Unnamed: 0', '_c0')
df.show(5)

+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+--------+
|modelMag_u|modelMag_g|modelMag_r|modelMag_i|modelMag_z|fiberMag_u|fiberMag_g|fiberMag_r|fiberMag_i|fiberMag_z|petroR50_r|petroR90_r|petroR50_z|petroR90_z|       r|       i|       z|     mmug|     mmgr|     mmri|     mmiz|     mfug|     mfgr|     mfri|     mfiz|redshift|
+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+--------+
|   23.2391|  21.44343|  19.67306|    18.944|  18.55688|  23.69614|  22.34922|  20.59603|  19.86871|  19.45047|  1.306018|  3.220677|  0.761024|  1.247238|19.67306|  18.944|18.55688|22

In [ ]:
from pyspark.sql.functions import monotonically_increasing_id
df = df.withColumn("objid", monotonically_increasing_id())
df.show(5)

+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+--------+-----+
|modelMag_u|modelMag_g|modelMag_r|modelMag_i|modelMag_z|fiberMag_u|fiberMag_g|fiberMag_r|fiberMag_i|fiberMag_z|petroR50_r|petroR90_r|petroR50_z|petroR90_z|       r|       i|       z|     mmug|     mmgr|     mmri|     mmiz|     mfug|     mfgr|     mfri|     mfiz|redshift|objid|
+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+--------+-----+
|   23.2391|  21.44343|  19.67306|    18.944|  18.55688|  23.69614|  22.34922|  20.59603|  19.86871|  19.45047|  1.306018|  3.220677|  0.761024|  1.247238|19.67306|  

In [ ]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols = ['modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i', 'modelMag_z', 'fiberMag_u', 'fiberMag_g', 
                                         'fiberMag_r', 'fiberMag_i', 'fiberMag_z', 'petroR50_r', 'petroR90_r', 'petroR50_z', 'petroR90_z', 
                                         'r', 'i', 'z', 'mmug', 'mmgr', 'mmri', 'mmiz', 'mfug', 'mfgr', 'mfri', 'mfiz'], outputCol = 'features')

In [ ]:
output = assembler.transform(df)
final_df = output.select('features', 'redshift')
final_df.show(3)

+--------------------+--------+
|            features|redshift|
+--------------------+--------+
|[23.2391,21.44343...|       0|
|[24.05348,21.2472...|       0|
|[26.32478,20.7405...|       0|
+--------------------+--------+
only showing top 3 rows



In [ ]:
train, test = final_df.randomSplit([0.7, 0.3])

In [ ]:
layers = [25, 24, 24, 8]

In [ ]:
classifier = MultilayerPerceptronClassifier(labelCol='redshift', featuresCol='features', maxIter=100, layers=layers, blockSize=128, seed=1234)

In [ ]:
start_time = time.time()
model = classifier.fit(train)
print("%s seconds" % (time.time() - start_time))

130.06255888938904 seconds


In [ ]:
result = model.transform(test)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

multi_evaluator = MulticlassClassificationEvaluator(labelCol = 'redshift', metricName = 'accuracy')
print('Multilayer Per:', multi_evaluator.evaluate(result))

Multilayer Per: 0.7920405663333618


In [ ]:
print('test data (weightedPrecision): ', multi_evaluator.setMetricName('weightedPrecision').evaluate(result))
print('test data (weightedRecall): ', multi_evaluator.setMetricName('weightedRecall').evaluate(result))

test data (weightedPrecision):  0.6869828794561721
test data (weightedRecall):  0.7920405663333618


#Cascading

In [ ]:
result2 = model.transform(final_df)

In [ ]:
result2.show(3)

+--------------------+--------+--------------------+--------------------+----------+
|            features|redshift|       rawPrediction|         probability|prediction|
+--------------------+--------+--------------------+--------------------+----------+
|[23.2391,21.44343...|       0|[6.37350090953163...|[0.93283416322037...|       0.0|
|[24.05348,21.2472...|       0|[6.20875618697094...|[0.92086308173206...|       0.0|
|[26.32478,20.7405...|       0|[6.72043791976690...|[0.95469379232940...|       0.0|
+--------------------+--------+--------------------+--------------------+----------+
only showing top 3 rows



In [ ]:
from pyspark.sql.functions import monotonically_increasing_id
df = df.withColumn("objid", monotonically_increasing_id())
df.show(5)

+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+--------+-----+
|modelMag_u|modelMag_g|modelMag_r|modelMag_i|modelMag_z|fiberMag_u|fiberMag_g|fiberMag_r|fiberMag_i|fiberMag_z|petroR50_r|petroR90_r|petroR50_z|petroR90_z|       r|       i|       z|     mmug|     mmgr|     mmri|     mmiz|     mfug|     mfgr|     mfri|     mfiz|redshift|objid|
+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+--------+-----+
|   23.2391|  21.44343|  19.67306|    18.944|  18.55688|  23.69614|  22.34922|  20.59603|  19.86871|  19.45047|  1.306018|  3.220677|  0.761024|  1.247238|19.67306|  

In [ ]:
from pyspark.sql.functions import monotonically_increasing_id
result2 = result2.withColumn("objid", monotonically_increasing_id())
result2.show(3)

+--------------------+--------+--------------------+--------------------+----------+-----+
|            features|redshift|       rawPrediction|         probability|prediction|objid|
+--------------------+--------+--------------------+--------------------+----------+-----+
|[23.2391,21.44343...|       0|[6.37350090953163...|[0.93283416322037...|       0.0|    0|
|[24.05348,21.2472...|       0|[6.20875618697094...|[0.92086308173206...|       0.0|    1|
|[26.32478,20.7405...|       0|[6.72043791976690...|[0.95469379232940...|       0.0|    2|
+--------------------+--------+--------------------+--------------------+----------+-----+
only showing top 3 rows



In [ ]:
print(df.count(), len(df.columns))
print(result2.count(), len(result2.columns))

new_df = df.join(result2, df.objid == result2.objid).select(df["*"],result2["prediction"])
#new_df = df.join(dt_predictions, df.objid == dt_predictions.objid)
new_df.show(5)

582231 27
582231 6
+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+--------+-----+----------+
|modelMag_u|modelMag_g|modelMag_r|modelMag_i|modelMag_z|fiberMag_u|fiberMag_g|fiberMag_r|fiberMag_i|fiberMag_z|petroR50_r|petroR90_r|petroR50_z|petroR90_z|       r|       i|       z|     mmug|     mmgr|     mmri|     mmiz|     mfug|     mfgr|     mfri|     mfiz|redshift|objid|prediction|
+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+--------+-----+----------+
|   23.2391|  21.44343|  19.67306|    18.944|  18.55688|  23.69614|  22.34922|  20.59603|  19.86871|  19.45047|  1

In [ ]:
assembler = VectorAssembler(inputCols = ['modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i', 'modelMag_z', 'fiberMag_u', 'fiberMag_g', 
                                         'fiberMag_r', 'fiberMag_i', 'fiberMag_z', 'petroR50_r', 'petroR90_r', 'petroR50_z', 'petroR90_z', 
                                         'r', 'i', 'z', 'mmug', 'mmgr', 'mmri', 'mmiz', 'mfug', 'mfgr', 'mfri', 'mfiz', 'prediction'], outputCol = 'features2')

In [ ]:
output = assembler.transform(new_df)
final_df = output.select('features2', 'redshift')
final_df.show(3)

+--------------------+--------+
|           features2|redshift|
+--------------------+--------+
|[23.2391,21.44343...|       0|
|[24.05348,21.2472...|       0|
|[25.42423,21.6029...|       0|
+--------------------+--------+
only showing top 3 rows



In [ ]:
train, test = final_df.randomSplit([0.7, 0.3])

In [ ]:
layers = [26, 24, 24, 8]

In [ ]:
classifier2 = MultilayerPerceptronClassifier(labelCol='redshift', featuresCol='features2', maxIter=100, layers=layers, blockSize=128, seed=1234)

In [ ]:
model2 = classifier2.fit(train)

In [ ]:
result2 = model2.transform(test)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

multi_evaluator = MulticlassClassificationEvaluator(labelCol = 'redshift', metricName = 'accuracy')
print('Decision Tree Accu:', multi_evaluator.evaluate(result2))

Decision Tree Accu: 0.7838357958592799


In [ ]:
print('test data (weightedPrecision): ', multi_evaluator.setMetricName('weightedPrecision').evaluate(result2))
print('test data (weightedRecall): ', multi_evaluator.setMetricName('weightedRecall').evaluate(result2))

test data (weightedPrecision):  0.746541651719919
test data (weightedRecall):  0.7838357958592798
